In [3]:
import pandas
import numpy 


"""Cho phép trực tiếp truy cập vào C-speed operations mà không mất phí cấp phát cho các mảng trung gian"""

randomer = numpy.random.RandomState(42)

x = randomer.rand(1000_000)
y = randomer.rand(1000_000)

%timeit x + y

1.95 ms ± 80.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
"""Phía trên nhanh hơn hẳn so với Python loop"""
%timeit numpy.fromiter((xi + yi for xi, yi in zip(x, y)), dtype = x.dtype, count = len(x))

225 ms ± 8.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
"""Tuy nhiên khi bạn tính toán các compound expression """
mask = (x > 0.5) & (y < 0.5)




In [6]:
"""Sở dĩ vậy vì NumPy đánh giá từng subexpression một, vì thế dòng code trên tương đương với"""
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

"""Và mọi intermediate step đều phải cấp phát bộ nhớ và nếu x, y lớn thì tính toán và bộ nhớ càng nặng"""
"""numpy cung cấp khả năng tính toán các loại compound expression element by element mà không cần cấp phát các mảng trung gian"""
import numexpr
maskNumExpr = numexpr.evaluate("(x > 0.5) & (y < 0.5)")

#Check xem kết quả trùng không nào
numpy.allclose(mask, maskNumExpr)

True

In [7]:
"""Pandas eval() và query() build trên top của Numexpr"""

"""pandas.eval() cho các efficient operations"""
nRows, nColumns = 100000, 100
randomer = numpy.random.RandomState(42)
df1, df2, df3, df4 = (
    pandas.DataFrame(randomer.rand(nRows, nColumns))
    for i in range(4)
)
%timeit df1 + df2 + df3 + df4 

106 ms ± 34.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
"""Bạn sẽ thấy pandas.eval() nhanh hơn 50% và sử dụng ít bộ nhớ hơn nhiều"""
%timeit pandas.eval("df1 + df2 + df3 + df4")

44.7 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
"""Đương nhiên đều cho kết quả tương tự rồi"""
numpy.allclose(
    df1 + df2 + df3 + df4,
    pandas.eval("df1 + df2 + df3 + df4")
)



True

In [11]:
"""Các operations có thể xài trong pandas.eval()"""

df1, df2, df3, df4, df5 = (
    pandas.DataFrame(
        randomer.randint(0, 1000, (100, 3))
    )
    for i in range(5)
)

"""Arithmetic operators"""
assert numpy.allclose(-df1 * df2 / (df3 + df4) - df5, pandas.eval("-df1 * df2 / (df3 + df4) - df5"))

In [12]:
"""Comparison operators"""

assert numpy.allclose((df1 < df2) & (df2 <= df3) & (df3 != df4), pandas.eval("(df1 < df2) & (df2 <= df3) & (df3 != df4)"))

In [13]:
"""Bitwise operators"""
assert numpy.allclose(
    (df1 < 0.5) & (df2 < 0.5) | (df3 < df4),
    pandas.eval("((df1 < 0.5) & (df2 < 0.5)) | (df3 < df4)")
)

In [14]:
"""Object attributes and indices"""

assert numpy.allclose(
    df2.T[0] + df3.iloc[1],
    pandas.eval("df2.T[0] + df3.iloc[1]")
)

In [9]:
import pandas
import numpy
"""Tuy nhiên, function calls, conditional statements, loops vẫn chưa được triển khai trong pandas.eval()"""
"""Nếu muốn thì bạn tự đi mà sử dụng Numexpr"""

"""Kinh dị hơn. dataframe.eval() cho phép bạn tham chiếu tới các cột chỉ bằng tên cột"""
df = pandas.DataFrame(
    randomer.rand(1000, 3),
    columns = ["A", "B", "C"]
)
numpy.allclose(
    pandas.eval("(df.A + df.B) / (df.C - 1)"),
    df.eval("(A + B) / (C - 1)")
)

True

In [16]:
"""Bạn còn có thể assignment trong pandas.eval()"""

df.eval("D = (A + B) / C", inplace = True)

NameError: name 'df' is not defined

In [11]:
"""Tham chiếu tới local variable trong dataframe.eval()"""

columnMean = df.mean(axis = 1)
numpy.allclose(
    df["A"] + columnMean,
    df.eval("A + @columnMean")
)

True

In [12]:
"""Bạn vẫn muốn tham chiếu cột bằng cách rút gọn và vẫn muốn có khả năng truy cập dataFrame ?"""
"""dataframe.query() là lựa chọn cho bạn"""

numpy.allclose(
    pandas.eval("df[(df.A < 0.5) & (df.B < 0.5)]"),
    df.query("A < 0.5 & B < 0.5")
)

True

In [ ]:
"""Kết luận: Bạn nên xài eval, query cho những dữ liệu kích thước lơn thực sự"""
"""Tuy nhiên cách tính toán truyền thống vẫn nhanh hơn với tập dữ liệu nhỏ đồng thời nó cũng dễ nhìn hơn"""
